In [1]:
import numpy as np
import queue
import copy
import matplotlib.pyplot as plt


In [2]:
# unit: hour
#即核酸检测有两个小时的检测窗口 eg. 9-11 a.m. 
#假设所有人都在这个区间进行检测👉后续可以优化
# simulation(hours)
#arrival rate (/hour) i.e. 2 per min
#service rate (/hour) 这里考虑改进前 即50s左右
#我们假设在正式检测开始之前，已经有人开始排队👉即这里的‘begin', 是一个0-30之间的随机整数
#number of server

begin = np.random.randint(0,30)
c = 3
total_time = 2
IAT_rate = 120
ST_rate = 72*c
rho = IAT_rate/ST_rate
print(begin)

28


In [3]:
# Initialize Parameters
# IAT  empty array for putting random number inter-arrival time
# St   ... for serving time rate
# AT: arrival times
# ST: serve time


qu = queue.Queue()
curr_process = None
IAT = []
ST = []
AT = []
wait_time = []
server_busy = False
list_wait = []
list_delay = []


In [4]:
# num_processes 即一共有多少个人在检测窗口里来做核酸检测

num_processes = int(np.random.poisson(IAT_rate)* total_time) 
total_people = num_processes + begin
num_processes_served = 0
print(num_processes)
print(begin)
print(total_people)

244
28
272


In [5]:
IAT_new = []

In [6]:
#Populate new Inter-Arrival-Times (IAT_new)
#即在正式检测开始之前到达的人的inter arrival time=0

for i in range(total_people):
    temp = np.random.exponential(1/IAT_rate)*60*60
    if i<begin:
        IAT_new.append(0)
        i=i+1
    else: 
        IAT_new.append(int(temp-temp%1))

print(IAT_new)


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 30, 34, 13, 2, 6, 123, 103, 12, 117, 12, 69, 25, 9, 0, 1, 8, 11, 0, 9, 13, 20, 25, 34, 19, 16, 42, 5, 21, 16, 18, 7, 44, 4, 40, 30, 16, 2, 16, 33, 92, 20, 26, 26, 88, 71, 13, 74, 5, 9, 10, 0, 20, 17, 43, 23, 19, 34, 33, 28, 15, 12, 10, 4, 45, 18, 74, 29, 26, 7, 6, 4, 5, 4, 38, 54, 19, 149, 51, 40, 12, 18, 8, 16, 31, 14, 57, 40, 4, 15, 26, 39, 13, 38, 8, 10, 64, 2, 9, 4, 16, 18, 10, 5, 24, 21, 13, 7, 12, 10, 17, 10, 41, 6, 9, 12, 17, 7, 3, 16, 3, 17, 3, 7, 4, 81, 64, 7, 9, 5, 40, 49, 41, 70, 53, 5, 18, 0, 19, 65, 5, 0, 62, 25, 1, 113, 8, 61, 4, 19, 21, 4, 38, 78, 6, 12, 21, 53, 26, 30, 53, 13, 21, 5, 22, 48, 122, 15, 31, 50, 9, 8, 47, 47, 22, 40, 81, 79, 79, 0, 22, 13, 39, 7, 58, 39, 20, 69, 102, 19, 23, 19, 15, 1, 65, 67, 32, 103, 10, 18, 32, 43, 27, 60, 58, 93, 19, 6, 26, 0, 7, 20, 4, 4, 0, 9, 17, 2, 36, 16, 66, 3, 57, 16, 25, 10, 7, 51, 13, 56, 32, 88, 26, 57, 32, 8, 23, 18, 0, 36, 21, 13, 12, 30]

In [7]:
# Populate Service-Times (ST) (where ST[i]!=0)

while not len(ST) == total_people:
    temp = np.random.exponential(1/ST_rate)*60*60
    if not int(temp- temp%1)<1:
        ST.append(int(temp - temp%1))

print(ST)

[55, 4, 9, 29, 15, 55, 18, 7, 28, 29, 18, 16, 6, 10, 11, 24, 16, 5, 7, 49, 4, 6, 16, 5, 8, 5, 2, 98, 34, 1, 8, 2, 24, 29, 24, 36, 1, 8, 13, 3, 1, 24, 10, 10, 40, 5, 3, 13, 7, 34, 15, 10, 15, 36, 7, 12, 8, 5, 10, 22, 2, 31, 19, 18, 19, 12, 20, 4, 13, 23, 6, 40, 8, 2, 26, 23, 11, 32, 15, 2, 12, 28, 35, 24, 85, 27, 26, 1, 22, 23, 18, 25, 1, 20, 1, 28, 5, 32, 2, 26, 3, 13, 4, 14, 1, 6, 4, 3, 31, 16, 13, 19, 1, 3, 22, 13, 15, 19, 54, 4, 3, 9, 15, 10, 28, 1, 6, 57, 51, 7, 1, 36, 6, 2, 3, 1, 14, 59, 28, 8, 30, 4, 15, 14, 7, 9, 7, 2, 30, 7, 10, 15, 17, 42, 2, 4, 6, 20, 4, 17, 7, 3, 10, 41, 27, 2, 6, 10, 13, 24, 34, 26, 50, 29, 14, 13, 5, 19, 11, 7, 40, 32, 6, 7, 7, 4, 29, 36, 1, 50, 7, 16, 11, 3, 48, 15, 16, 22, 4, 4, 31, 15, 35, 12, 3, 6, 14, 37, 9, 20, 23, 33, 5, 1, 31, 1, 25, 1, 59, 10, 5, 8, 2, 5, 5, 44, 40, 23, 18, 32, 2, 14, 40, 29, 24, 68, 4, 10, 49, 5, 78, 41, 8, 15, 11, 2, 25, 19, 16, 28, 6, 14, 2, 20, 42, 9, 15, 24, 14, 31, 58, 13, 8, 7, 42, 52, 12, 11, 62, 7, 3, 7]


In [8]:
# save a copy of ST

ST_copy = copy.deepcopy(ST)

In [9]:
AT_new = []

In [10]:
# Get new Arrival-Times (AT) from IAT starting at t=0 
# 生成到达时间队列
# and initialize Waiting-Times to 0

for i in range(total_people):
    if i<begin:
        AT_new.append(0)
        i=i+1
    else:
        AT_new.append(IAT_new[i]+AT_new[i-1])
    wait_time.append(0)

print(AT_new)


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 34, 68, 81, 83, 89, 212, 315, 327, 444, 456, 525, 550, 559, 559, 560, 568, 579, 579, 588, 601, 621, 646, 680, 699, 715, 757, 762, 783, 799, 817, 824, 868, 872, 912, 942, 958, 960, 976, 1009, 1101, 1121, 1147, 1173, 1261, 1332, 1345, 1419, 1424, 1433, 1443, 1443, 1463, 1480, 1523, 1546, 1565, 1599, 1632, 1660, 1675, 1687, 1697, 1701, 1746, 1764, 1838, 1867, 1893, 1900, 1906, 1910, 1915, 1919, 1957, 2011, 2030, 2179, 2230, 2270, 2282, 2300, 2308, 2324, 2355, 2369, 2426, 2466, 2470, 2485, 2511, 2550, 2563, 2601, 2609, 2619, 2683, 2685, 2694, 2698, 2714, 2732, 2742, 2747, 2771, 2792, 2805, 2812, 2824, 2834, 2851, 2861, 2902, 2908, 2917, 2929, 2946, 2953, 2956, 2972, 2975, 2992, 2995, 3002, 3006, 3087, 3151, 3158, 3167, 3172, 3212, 3261, 3302, 3372, 3425, 3430, 3448, 3448, 3467, 3532, 3537, 3537, 3599, 3624, 3625, 3738, 3746, 3807, 3811, 3830, 3851, 3855, 3893, 3971, 3977, 3989, 4010, 4063, 4089, 4119, 4

In [11]:
# simulate M/M/1 queue

for i in range(total_time*60*60):    
    if server_busy:
        for item in list(qu.queue):
            wait_time[item] = wait_time[item] + 1
        ST[curr_process] = ST[curr_process] - 1
        if ST[curr_process] == 0:
            server_busy = False
            num_processes_served = num_processes_served + 1

    for j in range(total_people):
        if i== AT_new[j]:
            qu.put(j)

    if not server_busy and not qu.empty():
        curr_process = qu.get()
        server_busy = True
        


In [12]:
# 总共等待时间
# 总共delay时间 = total time in the system 
# delay = waiting time + serve time
sum_wait = 0
sum_delay = 0

In [14]:
for i in range(num_processes_served):
    sum_wait = sum_wait + wait_time[i]
    sum_delay = sum_delay + wait_time[i] + ST_copy[i]
    
if num_processes_served == 0:
    list_wait.append(0)
    list_delay.append(0)
else:
    list_wait.append(sum_wait/(num_processes_served*60*60))  
    list_delay.append(sum_delay/(num_processes_served*60*60))


In [15]:
print(total_people)
print(begin)
print(num_processes)
print(num_processes_served)

272
28
244
272


In [16]:
#平均等待时间 in hour
print(sum_wait/(num_processes_served*60*60))
print(sum_delay/(num_processes_served*60*60))

# 平均等待时间 in minute
print(sum_wait/(num_processes_served*60))

0.027624591503267973
0.03259395424836601
1.6574754901960784


In [17]:
print((6000/worktime)*2)
print(salary/worktime)

NameError: name 'worktime' is not defined